In [10]:
from sympy.vector import CoordSys3D
from sympy import *
#
#  Curtis Problem 4.03

N = CoordSys3D('N')

r_x, r_y, r_z = symbols('r_x r_y r_z')
v_x, v_y, v_z = symbols('v_x v_y v_z')
mu = Symbol('mu')

# We are in the geocentric equatorial frame for this problem
r = r_x * N.i + r_y * N.j + r_z * N.k
v = v_x * N.i + v_y * N.j + v_z * N.k

values = {
    (r_x, 2500),
    (r_y, 16000),
    (r_z, 4000),
    (v_x, -3),
    (v_y, -1),
    (v_z, 5),
    ( mu, 3.986*10**5), # in units of km^3/s^2
}

# Equation 2.28 pg 69
h = r.cross(v)

# We can calculate the inclination i eq. 4.7 p.g. 191
i = acos(h.components[N.k]/h.magnitude())*180/pi

# Equation 2.40, rearranged
e = v.cross(h)/mu - r/r.magnitude()

# Equation 4.8, pg 191
Node = N.k.cross(h)

# We want the right ascension of the ascenting node
# Equation 4.9
Ω = acos(Node.components[N.i]/Node.magnitude())*180/pi
if(Node.components[N.j].subs(values).evalf() < 0):
    Ω = 360 - Ω

# to get argument of perigee equation 4.12 pg 192
ω = acos(Node.dot(e)/(Node.magnitude()*e.magnitude()))*180/pi
if(e.components[N.k].subs(values).evalf() < 0):
    ω = 360 - ω

# to get true enomoly we rearrange the definition of the dot product
# we also need to compute v_r
v_r = r.dot(v)/r.magnitude()

theta = acos(r.dot(e)/(r.magnitude()*e.magnitude()))*180/pi
if(v_r.subs(values).evalf() < 0):
    theta = 360 - theta

# Now to print out our results:
print("Eccentricity     e: ",e.magnitude().subs(values).evalf())
print("Angular Momentum h: ",h.magnitude().subs(values).evalf())
print("Inclination      i: ",i.subs(values).evalf())
print("Right Ascension  Ω: ",Ω.subs(values).evalf())
print("Arg. of Perigee  ω: ",ω.subs(values).evalf())
print("True Anomaly     θ: ",theta.subs(values).evalf())



# print(Node.magnitude().subs(values).evalf())

Eccentricity     e:  0.465758779923761
Angular Momentum h:  98623.0196252376
Inclination      i:  62.5255683737229
Right Ascension  Ω:  73.7397952916880
Arg. of Perigee  ω:  22.0805356392623
True Anomaly     θ:  353.600346745172


In [6]:
# now we want to go backwards; 
# i.e. compute radius and velocity given orbital elements

%reset -f
from sympy import *
from sympy.vector import CoordSys3D


e = Symbol('e')
h_p = Symbol('h_p')
r_p = Symbol('r_p')
i = Symbol('i')
Ω = Symbol('Ω')
ω = Symbol('ω')
theta = Symbol('theta')
mu = Symbol('mu')
r_earth = Symbol('r_earth')

values = {
    # (r_p, r_earth + h_p),
    (r_earth, 6378),     # km
    (h_p, 300),
    (mu, 3.986*10**5), # in units of km^3/s^2
    (e, 1.5),
    (i, 25),
    (Ω, 130),
    (ω, 115),
    (theta, 0),
}

r_p = r_earth + h_p

#
# determine angular momentum from orbit equation r = h**2 / mu / (1 + e * cos(theta))

h = sqrt( r_p * mu * (1 + e * cos(theta * pi / 180)))
print('h', h.subs(values).evalf())

#
# position and velocity in perifocal reference frame
#

# PQW = CoordSys3D('PQW')

r_P = r_p*cos(theta * pi / 180)# * PQW.i
r_Q = r_p*sin(theta  * pi / 180)# * PQW.j
r_W = 0# * PQW.k

v_P = -mu/h * sin(theta * pi / 180)# * PQW.i
v_Q = mu/h * ( e + cos(theta * pi / 180))# * PQW.j
v_W = 0# * PQW.k

r_PQW = Matrix( [r_P, r_Q, r_W] )
v_PQW = Matrix( [v_P, v_Q, v_W] )

#
# define Rotation Matrices
#

C_Ω = Matrix (
    [
        [ cos(Ω * pi / 180), sin(Ω * pi / 180), 0],
        [ -sin(Ω * pi / 180), cos(Ω * pi / 180), 0],
        [0,0,1]
    ]
)

A_i = Matrix(
    [
        [1,0,0],
        [0,cos(i*pi/180),sin(i*pi/180)],
        [0,-sin(i*pi/180),cos(i*pi/180)]
    ]
)

C_ω = Matrix (
    [
        [ cos(ω * pi / 180), sin(ω * pi / 180), 0],
        [ -sin(ω * pi / 180), cos(ω * pi / 180), 0],
        [0,0,1]
    ]
)

R = C_ω*A_i*C_Ω

r_ijk = R.transpose()*r_PQW
v_ijk = R.transpose()*v_PQW

print("positions",r_ijk.subs(values).evalf())
print("positions",v_ijk.subs(values).evalf())



h 81575.8971755751
